In [1]:
!pip install py2neo

In [2]:
%matplotlib inline

import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [3]:
uri = 'bolt://52.91.35.107:7687'
user = 'neo4j'
pwd = 'explanations-abettors-guys'

graph = Graph(uri, auth=('neo4j', pwd))
nodes_matcher = NodeMatcher(graph)

In [9]:
df = graph.run('MATCH (n:Node) RETURN n.name, n.node_labels, n.pptu_person, n.pptu_place, n.pptu_thing, n.pptu_unknown, n.word_vec, n.n2v_all_nodes').to_data_frame()
df.columns = ['name', 'node_labels', 'pptu_person', 'pptu_place', 
              'pptu_thing', 'pptu_unknown', 'word_vec', 'n2v_all_nodes']
df2 = df.fillna(0)
df2.head()

,name,node_labels,pptu_person,pptu_place,pptu_thing,pptu_unknown,word_vec,n2v_all_nodes
0,fire,[Subject],0.0,0.0,0.0,1,"[-0.7450879470830485, -0.20934578262995096, 0....","[-1.2516065835952759, -1.5556256771087646, 0.4..."
1,tenth final episode,[],0.0,0.0,0.0,1,"[-0.6770203057734492, -0.6750397022367982, -0....","[-1.1201648712158203, -1.4694828987121582, 0.2..."
2,season,[],0.0,0.0,1.0,1,"[0.06382344864086464, 0.13243938213195317, -0....","[-0.6323578357696533, -0.7736873626708984, 0.4..."
3,hbo medieval fantasy television series,[],0.0,0.0,0.0,1,"[0.46349801751917385, 0.9063730422859886, -0.4...","[-1.307327151298523, -1.4173364639282227, 0.21..."
4,thrones,[],0.0,0.0,1.0,1,"[-0.5009659925334025, -0.3552419633236281, 0.0...","[-0.8286945223808289, -0.3986292779445648, 0.5..."


In [11]:
df2.sample(10)

,name,node_labels,pptu_person,pptu_place,pptu_thing,pptu_unknown,word_vec,n2v_all_nodes
532,commonwealth realms,[],0.0,0.0,0.0,1,"[0.26180468430750325, -0.07367365816249594, 0....","[-3.9518115520477295, 0.6502695679664612, -0.7..."
200,finer tools,[],0.0,0.0,0.0,1,"[-0.482136507155988, -0.1636867369846431, 0.18...","[-3.0631442070007324, -0.19238506257534027, 0...."
555,sequel,[],0.0,0.0,0.0,1,"[0.6408584670175645, 0.10136704179488487, 0.23...","[-1.6076147556304932, 0.029984457418322563, 1...."
263,rhaenyra,[],1.0,0.0,0.0,1,"[-0.42872421452009846, -0.407687911993019, 0.7...","[-1.3586121797561646, -1.018962025642395, 0.17..."
133,morally ambiguous world,[],0.0,0.0,0.0,1,"[0.579950000090832, 0.6454295263730179, 0.9300...","[-0.6540330648422241, -0.9447450637817383, 0.5..."
51,belfast,[],0.0,0.0,0.0,1,"[0.8560839798821653, -0.7030274011848885, -0.7...","[-1.0050528049468994, -0.3465651869773865, 0.5..."
322,summer sea,[],0.0,0.0,0.0,1,"[0.5913672182699863, 0.6750450011355802, -0.28...","[-0.7886140942573547, -0.754361093044281, 0.90..."
309,screen actors guild awards,[],0.0,0.0,0.0,1,"[-0.9303459928514377, -0.28349311758760765, 0....","[-1.5691033601760864, -0.5073928833007812, 0.8..."
54,place,[],0.0,1.0,0.0,1,"[-0.2112994835244686, 0.9232192472352785, -0.7...","[-0.5697059035301208, -1.080573558807373, 0.64..."
96,drama series,[],0.0,0.0,0.0,1,"[0.7375030246490786, 0.4147727480707277, -0.98...","[-0.6066583395004272, -0.8179429173469543, 1.2..."
